In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import numpy.lib.stride_tricks as st

# --------------------------------------------------
# Paths and basic configuration
# --------------------------------------------------
PATH = "/content/drive/MyDrive/Datamining-TSC-Project/new_processed_data.parquet"
CFG = {
    "time_col": "time",
    "window": 36,        # sliding window length (hours)
    "trend_h": 12,       # recent hours for trend checks
    "ma_hours": [3, 6, 12],  # moving average windows
}

# --------------------------------------------------
# Angle utility functions
# --------------------------------------------------
def wrap360(x):
    # Wrap angles into [0, 360)
    return (x % 360.0 + 360.0) % 360.0

def angle_diff_deg(a, b):
    # Smallest signed angle difference a - b in degrees
    return (a - b + 180.0) % 360.0 - 180.0

def wave_dir_convert(old_wave_dir):
    # Convert wave direction to wind-direction convention
    return wrap360(270.0 - old_wave_dir)

# --------------------------------------------------
# Moving average feature generator
# --------------------------------------------------
def add_moving_averages(df, cols, ma_hours):
    # Add rolling mean features for selected columns
    for col in cols:
        for h in ma_hours:
            df[f"{col}_ma{h}"] = df[col].rolling(window=h, min_periods=h).mean()
    return df

# --------------------------------------------------
# Load and clean data
# --------------------------------------------------
df = pd.read_parquet(PATH)

df["time"] = pd.to_datetime(df["time"])
df = df.sort_values("time").drop_duplicates("time").reset_index(drop=True)

# Keep only relevant columns
df = df[
    [
        "time",
        "Wind speed",
        "Wind Direction",
        "Wave Period",
        "Wave Direction",
        "Wave Height",
        "Wave Power",
        "Pressure",
        "temperature",
        "Surge Height",
        "Total Water Level",
        "Wave Steepness",
    ]
].copy()

# Rename columns to short, consistent names
df.rename(
    columns={
        "Wind speed": "ws",
        "Wind Direction": "wd",
        "Wave Period": "tp",
        "Wave Direction": "wdir",
        "Wave Height": "hs",
        "Wave Power": "pwr",
        "Pressure": "mslp",
        "temperature": "temp",
        "Surge Height": "surge",
        "Total Water Level": "twl",
        "Wave Steepness": "steep",
    },
    inplace=True,
)

# --------------------------------------------------
# Wind–wave direction alignment features
# --------------------------------------------------
df["wdir"] = wave_dir_convert(df["wdir"].to_numpy(np.float32))

wd = df["wd"].to_numpy(np.float32)
wdir = df["wdir"].to_numpy(np.float32)

# Angle difference between wind and wave directions
dwd_deg = angle_diff_deg(wd, wdir).astype(np.float32)
dwd_rad = np.deg2rad(dwd_deg).astype(np.float32)

# Encode direction difference with sin/cos
df["dwd_sin"] = np.sin(dwd_rad).astype(np.float32)
df["dwd_cos"] = np.cos(dwd_rad).astype(np.float32)

# Drop raw direction columns
df.drop(columns=["wd", "wdir"], inplace=True)

# --------------------------------------------------
# Add moving average features
# --------------------------------------------------
ma_cols = [
    "hs", "ws", "pwr", "mslp",
    "temp", "surge", "twl", "steep", "tp"
]
df = add_moving_averages(df, ma_cols, CFG["ma_hours"])

# --------------------------------------------------
# Sliding window statistics
# --------------------------------------------------
W = CFG["window"]
H = CFG["trend_h"]

hs   = df["hs"].to_numpy(np.float32)
pwr  = df["pwr"].to_numpy(np.float32)
mslp = df["mslp"].to_numpy(np.float32)
ws   = df["ws"].to_numpy(np.float32)
dwd_cos = df["dwd_cos"].to_numpy(np.float32)

# Create rolling windows
hs_w   = st.sliding_window_view(hs,   W)
pwr_w  = st.sliding_window_view(pwr,  W)
mslp_w = st.sliding_window_view(mslp, W)
ws_w   = st.sliding_window_view(ws,   W)
dwd_cos_w = st.sliding_window_view(dwd_cos, W)

# Window-based severity metrics (mean + 2*std)
hs_metric36  = hs_w.mean(axis=1)  + 2.0 * hs_w.std(axis=1)
pwr_metric36 = pwr_w.mean(axis=1) + 2.0 * pwr_w.std(axis=1)

# --------------------------------------------------
# Train / validation / test split by time
# --------------------------------------------------
start_times = df["time"].iloc[:len(hs_metric36)].to_numpy()

train_mask = start_times < np.datetime64("2015-01-01")
val_mask   = (start_times >= np.datetime64("2015-01-01")) & (start_times < np.datetime64("2020-01-01"))
test_mask  = start_times >= np.datetime64("2020-01-01")

# --------------------------------------------------
# Percentile-based severity thresholds (train only)
# --------------------------------------------------
hs_p75, hs_p92, hs_p98, hs_p995 = np.percentile(
    hs_metric36[train_mask], [75, 92, 98, 99.5]
)
pwr_p75, pwr_p92, pwr_p98, pwr_p995 = np.percentile(
    pwr_metric36[train_mask], [75, 92, 98, 99.5]
)

# Map continuous values to severity classes
def severity(x, p75, p92, p98, p995):
    y = np.zeros_like(x, dtype=np.int8)
    y[(x >= p75) & (x < p92)]  = 1
    y[(x >= p92) & (x < p98)]  = 2
    y[(x >= p98) & (x < p995)] = 3
    y[(x >= p995)]             = 4
    return y

sev_hs  = severity(hs_metric36,  hs_p75,  hs_p92,  hs_p98,  hs_p995)
sev_pwr = severity(pwr_metric36, pwr_p75, pwr_p92, pwr_p98, pwr_p995)

# Base severity: worst of wave height or power
base = np.maximum(sev_hs, sev_pwr)

# --------------------------------------------------
# Trend-based reinforcement rules (train only)
# --------------------------------------------------
train_hours_mask = df["time"] < "2015-01-01"

hs_th   = np.percentile(hs[train_hours_mask],  92)
pwr_th  = np.percentile(pwr[train_hours_mask], 92)
ws_th   = np.percentile(ws[train_hours_mask],  92)
mslp_th = np.percentile(mslp[train_hours_mask], 20)
align_th = np.percentile(dwd_cos[train_hours_mask], 75)

# Count how many storm-like conditions persist in last H hours
cnt = (
    (hs_w[:, -H:]  >= hs_th).sum(axis=1)  >= 6
).astype(int) + (
    (pwr_w[:, -H:] >= pwr_th).sum(axis=1) >= 6
).astype(int) + (
    (ws_w[:, -H:]  >= ws_th).sum(axis=1)  >= 6
).astype(int) + (
    (mslp_w[:, -H:] <= mslp_th).sum(axis=1) >= 6
).astype(int) + (
    (dwd_cos_w[:, -H:] >= align_th).sum(axis=1) >= 4
).astype(int)

# Final labels (trend count can be used later if needed)
y = base.copy()

# --------------------------------------------------
# Sanity checks
# --------------------------------------------------
print("Columns:", df.columns.tolist())
print("Class dist:")
for name, m in [("train", train_mask), ("val", val_mask), ("test", test_mask)]:
    print(name, pd.Series(y[m]).value_counts(normalize=True).sort_index().to_dict())


Columns: ['time', 'ws', 'tp', 'hs', 'pwr', 'mslp', 'temp', 'surge', 'twl', 'steep', 'dwd_sin', 'dwd_cos', 'hs_ma3', 'hs_ma6', 'hs_ma12', 'ws_ma3', 'ws_ma6', 'ws_ma12', 'pwr_ma3', 'pwr_ma6', 'pwr_ma12', 'mslp_ma3', 'mslp_ma6', 'mslp_ma12', 'temp_ma3', 'temp_ma6', 'temp_ma12', 'surge_ma3', 'surge_ma6', 'surge_ma12', 'twl_ma3', 'twl_ma6', 'twl_ma12', 'steep_ma3', 'steep_ma6', 'steep_ma12', 'tp_ma3', 'tp_ma6', 'tp_ma12']
Class dist:
train {0: 0.725734690152414, 1: 0.1796986705606766, 2: 0.07084131909585957, 3: 0.017933740987496578, 4: 0.005791579203553284}
val {0: 0.7295774005111354, 1: 0.17819003285870755, 2: 0.06863818912011684, 3: 0.015950164293537787, 4: 0.007644213216502373}
test {0: 0.735375345217173, 1: 0.18250291009517722, 2: 0.060712573893593226, 3: 0.016889964165886836, 4: 0.0045192066281697215}


In [ ]:
import numpy as np
import numpy.lib.stride_tricks as st

from sklearn.metrics import f1_score, classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

# --------------------------------------------------
# Reproducibility
# --------------------------------------------------
SEED = 42
np.random.seed(SEED)

# Sliding window length
W = int(W)

# --------------------------------------------------
# Preprocessing options
# --------------------------------------------------
USE_STANDARDIZE = True
eps = 1e-6

# --------------------------------------------------
# XGBoost training configuration
# --------------------------------------------------
XGB_TRAIN_PARAMS = dict(
    objective="multi:softprob",   # multi-class probabilities
    eval_metric="mlogloss",
    num_class=None,               # set later
    eta=0.03,                     # learning rate
    max_depth=6,
    min_child_weight=2,
    subsample=0.8,
    colsample_bytree=0.8,
    lambda_=1.0,
    alpha=0.0,
    gamma=0.0,
    tree_method="hist",           # fast CPU histogram method
    seed=SEED,
)

NUM_BOOST_ROUND = 4000
EARLY_STOPPING_ROUNDS = 200
VERBOSE_EVAL = 100

# --------------------------------------------------
# Base features (hs/pwr excluded, temperature included)
# --------------------------------------------------
BASE_COLS = ["ws", "tp", "mslp", "surge", "twl", "steep", "temp", "dwd_sin", "dwd_cos"]

# --------------------------------------------------
# Moving average features (only if available)
# --------------------------------------------------
MA_HOURS = [3, 6, 12]
MA_COLS = []
for c in ["ws", "tp", "mslp", "surge", "twl", "steep", "temp"]:
    for h in MA_HOURS:
        name = f"{c}_ma{h}"
        if name in df.columns:
            MA_COLS.append(name)

# Final feature list
COLS = BASE_COLS + MA_COLS
print("Using features (hs/pwr excluded, temperature included):")
print(COLS)

# --------------------------------------------------
# Build sliding windows: (num_windows, W, num_features)
# --------------------------------------------------
arrs = [df[c].to_numpy(np.float32) for c in COLS]
X_list = [st.sliding_window_view(a, window_shape=W) for a in arrs]
X = np.stack(X_list, axis=-1).astype(np.float32)

# Align labels and masks with windows
y = np.asarray(y)[:len(X)]
train_mask = np.asarray(train_mask)[:len(X)]
val_mask   = np.asarray(val_mask)[:len(X)]
test_mask  = np.asarray(test_mask)[:len(X)]

# --------------------------------------------------
# Drop windows with NaN or inf values
# --------------------------------------------------
finite_mask = np.isfinite(X).all(axis=(1, 2))
if not finite_mask.all():
    print(f"Dropping {(~finite_mask).sum()} windows due to NaN/inf.")
    X = X[finite_mask]
    y = y[finite_mask]
    train_mask = train_mask[finite_mask]
    val_mask   = val_mask[finite_mask]
    test_mask  = test_mask[finite_mask]

# Train / val / test split
X_train, y_train = X[train_mask], y[train_mask]
X_val,   y_val   = X[val_mask],   y[val_mask]
X_test,  y_test  = X[test_mask],  y[test_mask]

n_classes = int(np.max(y_train)) + 1
print("Shapes:",
      "\n  X_train:", X_train.shape,
      "\n  X_val:  ", X_val.shape,
      "\n  X_test: ", X_test.shape,
      "\n  #classes:", n_classes)

# --------------------------------------------------
# Optional standardization (fit on TRAIN only)
# --------------------------------------------------
if USE_STANDARDIZE:
    # Compute stats over all time steps in TRAIN
    flat = X_train.reshape(-1, X_train.shape[-1])
    mu  = flat.mean(axis=0, keepdims=True)
    std = np.maximum(flat.std(axis=0, keepdims=True), eps)

    def standardize(Xn):
        return (Xn - mu) / std

    X_train = standardize(X_train).astype(np.float32)
    X_val   = standardize(X_val).astype(np.float32)
    X_test  = standardize(X_test).astype(np.float32)
    print("Standardization: ON (train-only).")
else:
    print("Standardization: OFF.")

# --------------------------------------------------
# Flatten windows for XGBoost (expects 2D input)
# --------------------------------------------------
Xtr = X_train.reshape(X_train.shape[0], -1)
Xva = X_val.reshape(X_val.shape[0], -1)
Xte = X_test.reshape(X_test.shape[0], -1)

print("Flattened shapes:",
      "\n  Xtr:", Xtr.shape,
      "\n  Xva:", Xva.shape,
      "\n  Xte:", Xte.shape)

# --------------------------------------------------
# Class weights and per-sample weights
# --------------------------------------------------
classes = np.unique(y_train)
cw = compute_class_weight(class_weight="balanced", classes=classes, y=y_train)
cw_map = {int(c): float(w) for c, w in zip(classes, cw)}
w_train = np.array([cw_map[int(yy)] for yy in y_train], dtype=np.float32)

print("Class weights:", cw_map)
print("Sample weights stats:", float(w_train.min()), float(w_train.mean()), float(w_train.max()))

# --------------------------------------------------
# Import XGBoost
# --------------------------------------------------
try:
    import xgboost as xgb
except Exception as e:
    raise ImportError(
        "xgboost could not be imported. It is usually available in Colab; otherwise `pip install xgboost`.\n"
        f"Original error: {e}"
    )

# Fill num_class dynamically
params = dict(XGB_TRAIN_PARAMS)
params["num_class"] = n_classes

# Build DMatrix objects
dtrain = xgb.DMatrix(Xtr, label=y_train.astype(np.int32), weight=w_train)
dval   = xgb.DMatrix(Xva, label=y_val.astype(np.int32))
dtest  = xgb.DMatrix(Xte, label=y_test.astype(np.int32))

evals = [(dtrain, "train"), (dval, "val")]

# --------------------------------------------------
# Train XGBoost with early stopping
# --------------------------------------------------
bst = xgb.train(
    params=params,
    dtrain=dtrain,
    num_boost_round=NUM_BOOST_ROUND,
    evals=evals,
    early_stopping_rounds=EARLY_STOPPING_ROUNDS,
    verbose_eval=VERBOSE_EVAL
)

best_ntree = getattr(bst, "best_iteration", None)
if best_ntree is not None:
    print("Best iteration:", int(best_ntree))

# --------------------------------------------------
# Prediction helpers
# --------------------------------------------------
def predict_labels(dmat):
    proba = bst.predict(dmat)
    # Multi-class: take argmax
    if proba.ndim == 1:
        return (proba > 0.5).astype(int)
    return np.argmax(proba, axis=1).astype(int)

def report_split(name, dmat, ys_true):
    yp = predict_labels(dmat)
    ys_true = ys_true.astype(int)
    print(f"\n{name}")
    print("F1-macro:", f1_score(ys_true, yp, average="macro"))
    print(classification_report(ys_true, yp, zero_division=0))
    print("Confusion matrix:")
    print(confusion_matrix(ys_true, yp))

# --------------------------------------------------
# Final reports
# --------------------------------------------------
report_split("TRAIN", dtrain, y_train)
report_split("VAL",   dval,   y_val)
report_split("TEST",  dtest,  y_test)


Using features (hs/pwr excluded, temperature included):
['ws', 'tp', 'mslp', 'surge', 'twl', 'steep', 'temp', 'dwd_sin', 'dwd_cos', 'ws_ma3', 'ws_ma6', 'ws_ma12', 'tp_ma3', 'tp_ma6', 'tp_ma12', 'mslp_ma3', 'mslp_ma6', 'mslp_ma12', 'surge_ma3', 'surge_ma6', 'surge_ma12', 'twl_ma3', 'twl_ma6', 'twl_ma12', 'steep_ma3', 'steep_ma6', 'steep_ma12', 'temp_ma3', 'temp_ma6', 'temp_ma12']
Dropping 11 windows due to NaN/inf.
Shapes: 
  X_train: (262957, 36, 30) 
  X_val:   (43824, 36, 30) 
  X_test:  (43813, 36, 30) 
  #classes: 5
Standardization: ON (train-only).
Flattened shapes: 
  Xtr: (262957, 1080) 
  Xva: (43824, 1080) 
  Xte: (43813, 1080)
Class weights: {0: 0.27557127511855173, 1: 1.1131868597070527, 2: 2.8230930270009127, 3: 11.151696352841391, 4: 34.53145108338805}
Sample weights stats: 0.2755712866783142 0.9999998807907104 34.53145217895508


/usr/local/lib/python3.12/dist-packages/xgboost/callback.py:386: UserWarning: [17:53:03] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "lambda_" } are not used.

  self.starting_round = model.num_boosted_rounds()


[0]	train-mlogloss:1.57355	val-mlogloss:1.56168
[100]	train-mlogloss:0.49149	val-mlogloss:0.38416
[200]	train-mlogloss:0.27407	val-mlogloss:0.22602
[300]	train-mlogloss:0.18382	val-mlogloss:0.17194
[400]	train-mlogloss:0.13694	val-mlogloss:0.14753
[500]	train-mlogloss:0.10721	val-mlogloss:0.13263
[600]	train-mlogloss:0.08626	val-mlogloss:0.12240
[700]	train-mlogloss:0.07160	val-mlogloss:0.11546
[800]	train-mlogloss:0.06046	val-mlogloss:0.11029
[900]	train-mlogloss:0.05144	val-mlogloss:0.10605
[1000]	train-mlogloss:0.04396	val-mlogloss:0.10266
[1100]	train-mlogloss:0.03800	val-mlogloss:0.10002
[1200]	train-mlogloss:0.03307	val-mlogloss:0.09790
[1300]	train-mlogloss:0.02898	val-mlogloss:0.09596
[1400]	train-mlogloss:0.02544	val-mlogloss:0.09462
[1500]	train-mlogloss:0.02258	val-mlogloss:0.09358
[1600]	train-mlogloss:0.02006	val-mlogloss:0.09245
[1700]	train-mlogloss:0.01792	val-mlogloss:0.09171
[1800]	train-mlogloss:0.01599	val-mlogloss:0.09097
[1900]	train-mlogloss:0.01435	val-mlogloss: